In [1]:
!pip install yfinance

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf

In [3]:
# Data Collection

# Define the tickers and the date range
tickers = ["AAPL", "NVDA", "GOOGL"]
start_date = "2020-01-01"
end_date = "2023-01-01"

# Download the historical stock price data
data = yf.download(tickers, start=start_date, end=end_date)["Adj Close"]

# Calculate daily returns
daily_returns = data.pct_change().dropna()

# Calculate average daily returns
avg_daily_returns = daily_returns.mean()
print("Average Daily Returns:")
print(avg_daily_returns)

# Calculate daily return standard deviations
std_daily_returns = daily_returns.std()
print("Daily Return Standard Deviations:")
print(std_daily_returns)

# Calculate the correlation matrix
correlation_matrix = daily_returns.corr()
print("Correlation Matrix:")
print(correlation_matrix)

[*********************100%***********************]  3 of 3 completed

Average Daily Returns:
Ticker
AAPL     0.001024
GOOGL    0.000573
NVDA     0.001806
dtype: float64
Daily Return Standard Deviations:
Ticker
AAPL     0.023266
GOOGL    0.021748
NVDA     0.035262
dtype: float64
Correlation Matrix:
Ticker      AAPL     GOOGL      NVDA
Ticker                              
AAPL    1.000000  0.730437  0.713038
GOOGL   0.730437  1.000000  0.713155
NVDA    0.713038  0.713155  1.000000


In [4]:
# Define Chromosome Design

# Number of stocks in the portfolio
N = len(tickers)

# Example of a chromosome: weights for each stock
chromosome = np.random.dirichlet(np.ones(N), size=1)[0]

# Display the chromosome
print("Chromosome (Weights):", chromosome)

Chromosome (Weights): [0.92152639 0.07100795 0.00746566]


In [5]:
def fitness(chromosome, expected_returns, cov_matrix, alpha=1.0, beta=1.0):
    # Calculate the portfolio return
    portfolio_return = np.dot(chromosome, expected_returns)

    # Calculate the portfolio risk (standard deviation of the portfolio)
    portfolio_risk = np.sqrt(np.dot(chromosome.T, np.dot(cov_matrix, chromosome)))

    # Fitness function: balance between return and risk
    return alpha * portfolio_return - beta * portfolio_risk

expected_returns = np.random.rand(N) * 0.2  # Random expected returns for each stock
cov_matrix = np.random.rand(N, N)
cov_matrix = (cov_matrix + cov_matrix.T) / 2  # Symmetrize the covariance matrix
np.fill_diagonal(cov_matrix, np.random.rand(N) * 0.15)  # Fill diagonal with random variances

# Calculate fitness for the example chromosome
fitness_value = fitness(chromosome, expected_returns, cov_matrix)
print("Fitness Value:", fitness_value)

Fitness Value: -0.28000511216545576


In [6]:
# Initialize Population

# Population size
POP_SIZE = 100

# Initialize the population with random chromosomes
population = np.random.dirichlet(np.ones(N), size=POP_SIZE)

# Display the initial population
print("Initial Population:")
print(population)

Initial Population:
[[0.28523882 0.26157748 0.4531837 ]
 [0.07625031 0.81670722 0.10704247]
 [0.38090956 0.50370384 0.1153866 ]
 [0.34575673 0.50097944 0.15326383]
 [0.59095793 0.06758377 0.3414583 ]
 [0.5079121  0.14940467 0.34268323]
 [0.43635015 0.36230252 0.20134732]
 [0.3853656  0.35226002 0.26237437]
 [0.07904987 0.82862015 0.09232998]
 [0.42535476 0.01775923 0.55688602]
 [0.01449917 0.05469574 0.9308051 ]
 [0.12919972 0.55476827 0.31603201]
 [0.27885057 0.05255346 0.66859597]
 [0.34721836 0.52880156 0.12398008]
 [0.02489349 0.7772176  0.19788891]
 [0.24748606 0.33649311 0.41602083]
 [0.82864971 0.06181933 0.10953096]
 [0.30769923 0.48997338 0.20232739]
 [0.12352979 0.71934399 0.15712622]
 [0.33004924 0.33033625 0.33961451]
 [0.16122072 0.45171212 0.38706717]
 [0.09258148 0.80189333 0.1055252 ]
 [0.21640583 0.10649978 0.67709439]
 [0.04537685 0.07528728 0.87933586]
 [0.56610029 0.3448201  0.08907961]
 [0.32077417 0.03335939 0.64586645]
 [0.11610841 0.82867466 0.05521692]
 [0.2259

In [7]:
# Selection Process

# Calculate fitness values for the entire population
fitness_values = np.array([fitness(chrom, expected_returns, cov_matrix) for chrom in population])

# Roulette Wheel Selection
def roulette_wheel_selection(population, fitness_values):
    # Calculate the total fitness
    total_fitness = np.sum(fitness_values)

    # Calculate the probability of selection for each chromosome
    selection_probs = fitness_values / total_fitness

    # Calculate the cumulative probability distribution
    cumulative_probs = np.cumsum(selection_probs)

    # Select new population based on cumulative probability distribution
    new_population = []
    for _ in range(POP_SIZE):
        r = np.random.rand()
        selected_idx = np.where(cumulative_probs >= r)[0][0]
        new_population.append(population[selected_idx].copy())

    return np.array(new_population)

# Perform selection to create a new population
new_population = roulette_wheel_selection(population, fitness_values)

# Display the new population
print("New Population after Selection:")
print(new_population)

New Population after Selection:
[[0.41618584 0.41318535 0.17062882]
 [0.35331162 0.36528967 0.2813987 ]
 [0.19849621 0.30369422 0.49780957]
 [0.54575695 0.20549245 0.2487506 ]
 [0.38090956 0.50370384 0.1153866 ]
 [0.37494118 0.28955038 0.33550844]
 [0.06538669 0.14026721 0.7943461 ]
 [0.58649676 0.21473227 0.19877097]
 [0.44865391 0.28069368 0.27065241]
 [0.06033821 0.47343002 0.46623177]
 [0.24748606 0.33649311 0.41602083]
 [0.82864971 0.06181933 0.10953096]
 [0.38090956 0.50370384 0.1153866 ]
 [0.22594994 0.48661871 0.28743135]
 [0.45704368 0.47804994 0.06490638]
 [0.37494118 0.28955038 0.33550844]
 [0.56610029 0.3448201  0.08907961]
 [0.23640419 0.53144881 0.232147  ]
 [0.84011018 0.13016845 0.02972137]
 [0.03275468 0.81157056 0.15567476]
 [0.24748606 0.33649311 0.41602083]
 [0.52542299 0.0616825  0.41289451]
 [0.20702159 0.64648045 0.14649796]
 [0.02489349 0.7772176  0.19788891]
 [0.34575673 0.50097944 0.15326383]
 [0.06538669 0.14026721 0.7943461 ]
 [0.01449917 0.05469574 0.930805

In [8]:
def crossover(population, cross_rate=0.8):
    new_population = population.copy()
    for i in range(0, POP_SIZE, 2):
        if np.random.rand() < cross_rate:
            crossover_point = np.random.randint(1, N)  # Single-point crossover
            parent1 = population[i].copy()
            parent2 = population[i+1].copy()
            new_population[i, crossover_point:] = parent2[crossover_point:]
            new_population[i+1, crossover_point:] = parent1[crossover_point:]
    return new_population

# Perform crossover to generate offspring
offspring_population = crossover(new_population)

# Display the offspring population
print("Offspring Population after Crossover:")
print(offspring_population)

Offspring Population after Crossover:
[[0.41618584 0.36528967 0.2813987 ]
 [0.35331162 0.41318535 0.17062882]
 [0.19849621 0.20549245 0.2487506 ]
 [0.54575695 0.30369422 0.49780957]
 [0.38090956 0.28955038 0.33550844]
 [0.37494118 0.50370384 0.1153866 ]
 [0.06538669 0.21473227 0.19877097]
 [0.58649676 0.14026721 0.7943461 ]
 [0.44865391 0.28069368 0.46623177]
 [0.06033821 0.47343002 0.27065241]
 [0.24748606 0.33649311 0.41602083]
 [0.82864971 0.06181933 0.10953096]
 [0.38090956 0.50370384 0.28743135]
 [0.22594994 0.48661871 0.1153866 ]
 [0.45704368 0.28955038 0.33550844]
 [0.37494118 0.47804994 0.06490638]
 [0.56610029 0.3448201  0.08907961]
 [0.23640419 0.53144881 0.232147  ]
 [0.84011018 0.13016845 0.02972137]
 [0.03275468 0.81157056 0.15567476]
 [0.24748606 0.0616825  0.41289451]
 [0.52542299 0.33649311 0.41602083]
 [0.20702159 0.64648045 0.14649796]
 [0.02489349 0.7772176  0.19788891]
 [0.34575673 0.50097944 0.7943461 ]
 [0.06538669 0.14026721 0.15326383]
 [0.01449917 0.19656496 0.

In [9]:
def mutation(population, mutate_rate=0.05):
    for i in range(POP_SIZE):
        if np.random.rand() < mutate_rate:
            mutation_vector = np.random.dirichlet(np.ones(N), size=1)[0]
            population[i] = mutation_vector
    return population

# Apply mutation to the offspring population
mutated_population = mutation(offspring_population)

# Display the mutated population
print("Mutated Population after Mutation:")
print(mutated_population)

Mutated Population after Mutation:
[[0.41618584 0.36528967 0.2813987 ]
 [0.35331162 0.41318535 0.17062882]
 [0.19849621 0.20549245 0.2487506 ]
 [0.54575695 0.30369422 0.49780957]
 [0.38090956 0.28955038 0.33550844]
 [0.37494118 0.50370384 0.1153866 ]
 [0.06538669 0.21473227 0.19877097]
 [0.58649676 0.14026721 0.7943461 ]
 [0.44865391 0.28069368 0.46623177]
 [0.06033821 0.47343002 0.27065241]
 [0.24748606 0.33649311 0.41602083]
 [0.82864971 0.06181933 0.10953096]
 [0.38090956 0.50370384 0.28743135]
 [0.22594994 0.48661871 0.1153866 ]
 [0.45704368 0.28955038 0.33550844]
 [0.37494118 0.47804994 0.06490638]
 [0.56610029 0.3448201  0.08907961]
 [0.23640419 0.53144881 0.232147  ]
 [0.84011018 0.13016845 0.02972137]
 [0.35260477 0.56511272 0.08228251]
 [0.24748606 0.0616825  0.41289451]
 [0.52542299 0.33649311 0.41602083]
 [0.20702159 0.64648045 0.14649796]
 [0.02489349 0.7772176  0.19788891]
 [0.34575673 0.50097944 0.7943461 ]
 [0.06538669 0.14026721 0.15326383]
 [0.01449917 0.19656496 0.549

In [10]:
# Termination Condition

# Define the maximum number of generations
GEN_MAX = 100

# Initialize the generation counter
generation = 0

# Run the genetic algorithm until the termination condition is met
while generation < GEN_MAX:
    # Calculate fitness values for the current population
    fitness_values = np.array([fitness(chrom, expected_returns, cov_matrix) for chrom in population])

    # Perform selection to create a new population
    population = roulette_wheel_selection(population, fitness_values)

    # Perform crossover to generate offspring
    population = crossover(population)

    # Apply mutation to the offspring population
    population = mutation(population)

    # Increment the generation counter
    generation += 1

# Calculate the final fitness values
final_fitness_values = np.array([fitness(chrom, expected_returns, cov_matrix) for chrom in population])

# Find the best chromosome in the final population
best_idx = np.argmax(final_fitness_values)
best_chromosome = population[best_idx]

# Display the best chromosome and its fitness value
print("Best Chromosome:", best_chromosome)
print("Best Fitness Value:", final_fitness_values[best_idx])

Best Chromosome: [0.05913345 0.15131927 0.78954729]
Best Fitness Value: -0.376374799910722


In [11]:
# Output Results

# Calculate the best chromosome's return
best_return = np.dot(best_chromosome, expected_returns)

# Calculate the best chromosome's risk
best_risk = np.sqrt(np.dot(best_chromosome.T, np.dot(cov_matrix, best_chromosome)))

# Output the results
print("Best Chromosome's Return:", best_return)
print("Best Chromosome's Risk:", best_risk)
print("Selected Stocks (Weights):", best_chromosome)

Best Chromosome's Return: 0.09700201567158401
Best Chromosome's Risk: 0.473376815582306
Selected Stocks (Weights): [0.05913345 0.15131927 0.78954729]
